 
# Import dependencies


In [4]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
%matplotlib qt
import os
import pandas as pd
import glob
import csv

from mne import pick_types, Annotations, create_info, find_events, Epochs
from mne.channels import make_standard_montage
from mne.io import RawArray
from mne.epochs import concatenate_epochs
from mne.decoding import CSP

from scipy.signal import welch
from mne.viz.topomap import _prepare_topo_plot, plot_topomap
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
import pickle as pkl

from sklearn.metrics import log_loss

## Train a neural network

###### We'll use series 1 through 6 for training and series 7 and 8 for validating the deep learning model

In [ ]:
# get the data file names for a all subject, series 1 through 8 for normalization
series = list(range(1,9))
file = f'grasp-and-lift-eeg-detection/train/subj*_series{series}_data.csv'
dirpath = os.path.join(os.getcwd(),file)
glob_object = glob.glob(dirpath)

In [ ]:
# load all the data and calculate the mean and standard deviation for each channel
data_all = np.empty((32,0))
for file in glob_object:
    data_df = pd.read_csv(file).drop(['id'], axis=1)
    data_np = data_df.to_numpy().transpose()
    data_all = np.concatenate((data_all,data_np),axis=1)
MEAN = data_all.mean(1).reshape(32,1)
ST_DEV = data_all.std(1).reshape(32,1)
maximum = np.amax(data_all, axis = 1).reshape(32,1)
minimum = np.amin(data_all, axis = 1).reshape(32,1)

In [ ]:
pkl.dump(MEAN, open("MEAN.pkl", "wb"))
pkl.dump(ST_DEV, open("ST_DEV.pkl", "wb"))
pkl.dump(maximum, open("maximum.pkl", "wb"))
pkl.dump(minimum, open("minimum.pkl", "wb"))

In [29]:
MEAN = pkl.load( open( "MEAN.pkl", "rb" ) )
ST_DEV = pkl.load( open( "ST_DEV.pkl", "rb" ) )
maximum = pkl.load( open( "maximum.pkl", "rb" ) )
minimum = pkl.load( open( "minimum.pkl", "rb" ) )

In [ ]:
# get the data file names for a all subject, series 1 through 6 for generating the training data
series = list(range(1,7))
file = f'grasp-and-lift-eeg-detection/train/subj*_series{series}_data.csv'
dirpath = os.path.join(os.getcwd(),file)
glob_object = glob.glob(dirpath)

In [17]:
batch_size = 32
win_length = 150

In [3]:
def reshape_data(data,events,batch_size,win_length):
    batches = data.shape[1]//(win_length*batch_size)  
    
    data_out = np.empty((batches,batch_size,32,win_length))
    events_out = np.empty((batches,batch_size,6))
    
    data_temp = np.empty((batch_size,32,win_length))
    events_temp = np.empty((batch_size,6))
    
    batch_length = batch_size*win_length
    
    for k in range(batches):
        for i in range(batch_size):
            data_temp[i,:,:] = data[:,(k*batch_length)+(i*win_length):(k*batch_length)+(i+1)*win_length]
            events_temp[i] = events[:,(k*batch_length)+(i+1)*win_length-1]
        data_out[k,:,:,:] = data_temp
        events_out[k,:,:] = events_temp
    return (data_out, events_out)

In [ ]:
def generate_data(glob_object):
    
    X = np.empty((0,batch_size,32,win_length))
    Y = np.empty((0,batch_size,6))

    for file in glob_object:

        data_df = pd.read_csv(file).drop(['id'], axis=1)
        data_np = data_df.to_numpy().transpose()
        # threshold data so it can fit in batches
        thresh = data_np.shape[1] - (data_np.shape[1] % (batch_size*win_length))
        data_np = data_np[:,:thresh]
        #normalize the data
        data_np = data_np - minimum
        data_np = data_np  / (maximum - minimum)     
        file = file.replace('data','events')
        events_df = pd.read_csv(file).drop(['id'], axis=1)
        events_np = events_df.to_numpy().transpose()
        events_np = events_np[:,:thresh]   

        data, events = reshape_data(data_np,events_np,batch_size,win_length)

        X = np.concatenate((X,data),axis=0)
        Y = np.concatenate((Y,events),axis=0)

    return (X, Y)

In [ ]:
X_train, Y_train = generate_data(glob_object)

In [4]:
print(X_train.shape)
print(Y_train.shape)

(3046, 32, 32, 150)
(3046, 32, 6)


In [ ]:
print(np.amax(X_train[:,:,:,:]))
print(np.amin(X_train[:,:,:,:]))

In [ ]:
pkl.dump(X_train, open("X_train.pkl", "wb"))
pkl.dump(Y_train, open("Y_train.pkl", "wb"))

In [5]:
X_train = pkl.load( open( "X_train.pkl", "rb" ) )
Y_train = pkl.load( open( "Y_train.pkl", "rb" ) )

In [ ]:
# get the data file names for a all subject, series 7 and 8 for building the validation data
series = [7,8]
file = f'grasp-and-lift-eeg-detection/train/subj*_series{series}_data.csv'
dirpath = os.path.join(os.getcwd(),file)
glob_object = glob.glob(dirpath)

In [ ]:
X_valid, Y_valid = generate_data(glob_object)

In [16]:
print(X_valid.shape)
print(Y_valid.shape)

(646, 32, 32, 150)
(646, 32, 6)


In [ ]:
print(np.amax(X_valid[:,:,:,:]))
print(np.amin(X_valid[:,:,:,:]))

In [ ]:
pkl.dump(X_valid, open("X_valid.pkl", "wb"))
pkl.dump(Y_valid, open("Y_valid.pkl", "wb"))

In [9]:
X_valid = pkl.load( open( "X_valid.pkl", "rb" ) )
Y_valid = pkl.load( open( "Y_valid.pkl", "rb" ) )

In [7]:
def binarize_result(arr, batch_size):
    return torch.where(arr > 0.2, torch.ones(batch_size,6), torch.zeros(batch_size,6)).numpy()

In [8]:
def calculate_precision(model,X,Y):
    tp = 0
    fp = 0
    total = 0
    model.eval()
    for i in range(X.shape[0]):
        y_hat = model(torch.tensor(X[i,:,:,:]))
        y_hat = binarize_result(y_hat,batch_size)
        y = Y[i,:,:]
        for j in range(batch_size):
            if (sum(y[j]) != 0):
                total += 1
            if (sum(y[j]) == 0 and sum(y_hat[j]) != 0 ):
                fp += 1
            if (sum(y[j]) != 0 and np.array_equal(y[j],y_hat[j])):
                tp += 1   
    print(" tp:", tp, " fp:", fp)
    if (tp+fp) == 0:
        return 0
    else:
        return tp/(tp+fp)

In [21]:
def calculate_valid_loss(model,X,Y):
    model.eval()
    total_loss = 0.0
    for i in range(X.shape[0]):
        y_hat = model(torch.tensor(X[i,:,:,:])).detach().numpy()
        y = Y[i,:,:] 
        total_loss += log_loss(y,y_hat)
    return total_loss

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

GeForce GTX 1070
True


In [10]:
class OneDimensionalConvolution(torch.nn.Module):
    
    def __init__(self, batch_size):
        super(OneDimensionalConvolution, self).__init__()
        
        self.batch_size = batch_size
        
        self.c1 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1)
        self.c2 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, stride=1)
        self.m1 = nn.MaxPool1d(3, stride=2)     
        
        self.c3 = nn.Conv1d(in_channels=8, out_channels=32, kernel_size=3, stride=1)
        self.c4 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1) 
        self.m2 = nn.MaxPool1d(3, stride=2)   
        
        self.c5 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=3, stride=1)
        self.c6 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1) 
        self.m3 = nn.MaxPool1d(3, stride=2)           
        
        self.c7 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.c8 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1) 
        self.m4 = nn.MaxPool1d(3, stride=2)    
               
        self.d1 = nn.Dropout(p=0.1)     
        self.l1 = nn.Linear(128,64)
        
        self.d2 = nn.Dropout(p=0.1)
        self.l2 = nn.Linear(64,64)
        
        self.l3 = nn.Linear(64,6)

        
        
    def forward(self, x): 
        x = self.m1(self.c2(self.c1(x))) 
        x = self.m2(self.c4(self.c3(x)))
        x = self.m3(self.c6(self.c5(x)))     
        x = self.m4(self.c8(self.c7(x)))
        
        x = self.d1(x)
        x = x.reshape(self.batch_size,-1)
        x = self.l1(x)
        x = F.relu(x)
        
        x = self.d2(x)
        x = self.l2(x)
        x = F.relu(x)
        
        x = self.l3(x)
        x = torch.sigmoid(x)


        return x
    

 

In [18]:
model = OneDimensionalConvolution(batch_size).double()

In [43]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [44]:
no_batches = X_train.shape[0]

for epoch in range(20):   
    
    running_loss = 0.0
    
    for k in range(no_batches):
        optimizer.zero_grad()
        model.train()
        y = model(torch.tensor(X_train[k,:,:,:],requires_grad=True))
        y_hat = torch.tensor(Y_train[k,:,:])
        loss = criterion(y, y_hat)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print(f'epoch: {epoch}, running loss: {running_loss}, validation loss: {calculate_valid_loss(model,X_valid,Y_valid)}')
    #print(calculate_precision(model,X_valid,Y_valid))
print('Finished Training')

epoch: 0, running loss: 330.8370005353484, validation loss: 269.48424190304326
epoch: 1, running loss: 330.4646610085767, validation loss: 269.48311995448404
epoch: 2, running loss: 330.710903397521, validation loss: 269.4824818592175
epoch: 3, running loss: 330.76603404552867, validation loss: 269.48164498390736
epoch: 4, running loss: 330.6543751529954, validation loss: 269.4812012527968
epoch: 5, running loss: 330.5838053415779, validation loss: 269.480296991482
epoch: 6, running loss: 330.4593879159964, validation loss: 269.47982792301906
epoch: 7, running loss: 330.4431805613584, validation loss: 269.47944724915965
epoch: 8, running loss: 330.4432558034193, validation loss: 269.4790275735551
epoch: 9, running loss: 330.61049163700477, validation loss: 269.4787642934822
epoch: 10, running loss: 330.43215315898175, validation loss: 269.47838695993386
epoch: 11, running loss: 330.2889846912506, validation loss: 269.4777754105493
epoch: 12, running loss: 330.4215937184031, validation 

#################################################### 

In [24]:
dirpath = os.path.join(os.getcwd(),'model_SGD_minmax_32')
torch.save(model.state_dict(), dirpath)

In [26]:
model = OneDimensionalConvolution(batch_size).double()
model.load_state_dict(torch.load(os.path.join(os.getcwd(),'model_SGD_minmax_32_1')))
#model.eval()
#calculate_precision(model, X_valid, Y_valid)

<All keys matched successfully>

In [27]:
model

OneDimensionalConvolution(
  (c1): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
  (c2): Conv1d(16, 8, kernel_size=(3,), stride=(1,))
  (m1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c3): Conv1d(8, 32, kernel_size=(3,), stride=(1,))
  (c4): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
  (m2): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c5): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
  (c6): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
  (m3): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c7): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (c8): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
  (m4): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout(p=0.1, inplace=False)
  (l1): Linear(in_features=128, out_features=64, bias=True)
  (d2): Dropout(p=0.1, inplace=False)
  (l2): Linear(in_features=64, out_features=64, bias=True)
  (l3): Linear(i

In [33]:
# get the test data and arrange the data to have the same order as in the submission file
file = f'grasp-and-lift-eeg-detection/test/subj*_series9_data.csv'
dirpath = os.path.join(os.getcwd(),file)
glob_object1 = glob.glob(dirpath)

file = f'grasp-and-lift-eeg-detection/test/subj*_series10_data.csv'
dirpath = os.path.join(os.getcwd(),file)
glob_object2 = glob.glob(dirpath)

glob_object1.sort()
glob_object2.sort()

glob_object1 = glob_object1[3:]+glob_object1[:3]
glob_object2 = glob_object2[3:]+glob_object2[:3]

glob_object = []

for i in range(12):
    glob_object.append(glob_object1[i])
    glob_object.append(glob_object2[i])
    
for f in glob_object:
    print(f)

/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj1_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj1_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj2_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj2_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj3_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj3_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj4_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj4_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj5_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj5_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj6_series9_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/subj6_series10_data.csv
/home/cinetic-vr/EEG/grasp-and-lift-eeg-detection/test/sub

In [34]:
def reshape_test_data(data,batch_size,win_length):
    batches = data.shape[1]//(win_length*batch_size)  
    
    data_out = np.empty((batches,batch_size,32,win_length))   
    data_temp = np.empty((batch_size,32,win_length))
    batch_length = batch_size*win_length
    
    for k in range(batches):
        for i in range(batch_size):
            data_temp[i,:,:] = data[:,(k*batch_length)+(i*win_length):(k*batch_length)+(i+1)*win_length]
        data_out[k,:,:,:] = data_temp
    return data_out

In [35]:
X_test = np.empty((0,batch_size,32,win_length))

for file in glob_object:

    data_df = pd.read_csv(file).drop(['id'], axis=1)
    data_np = data_df.to_numpy().transpose()
    # threshold data so it can fit in batches
    thresh = data_np.shape[1] - (data_np.shape[1] % (batch_size*win_length))
    data_np = data_np[:,:thresh]
    #normalize the data
    data_np = (data_np - minimum) / (maximum - minimum)
    data = reshape_test_data(data_np,batch_size,win_length)

    X_test = np.concatenate((X_test,data),axis=0)
 

 

In [36]:
X_test.shape

(642, 32, 32, 150)

In [37]:
pkl.dump(X_test, open("X_test.pkl", "wb"))